In [7]:
import subprocess
from pyspark.sql.types import IntegerType
from functools import reduce
from pyspark import SparkConf
from pyspark.sql import DataFrame,SparkSession
from pyspark.sql.functions import sum
conf = SparkConf().setMaster("yarn").setAppName("test")
spark=SparkSession.builder.config(conf=conf).getOrCreate()

p = subprocess.Popen("hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/aggre| awk '{print $8}' | awk 'NR!=1' ",
shell=True,
stdout=subprocess.PIPE,
stderr=subprocess.STDOUT)


In [8]:
def hdfs_file_read(file):
    rdd = spark.read.csv(file, header = True)
    # rdd=rdd.drop("axlespacings", "axleweights", "numberofaxles","validitycode","duration","temperature","weight","hour","minute","second","millisecond")
    return rdd

In [9]:
res = list()
for line in p.stdout.readlines():
    file = line.decode().strip()
    rt = hdfs_file_read(file)
    rt= rt.withColumn("VehicleCount", rt["VehicleCount"].cast(IntegerType()))
    rt.show()
    dff= rt.groupBy("class").sum("VehicleCount")
    dff.show()
    dff.printSchema()
    dff=dff.withColumnRenamed("sum(VehicleCount)", "TotalCount")
    res.append(dff)

union_df = reduce(DataFrame.unionAll, res)
union_df=union_df.groupBy("class").avg("TotalCount").sort('class')
union_df.show()


+------------+-----+----+-----+---+------------+
|       cosit|class|year|month|day|VehicleCount|
+------------+-----+----+-----+---+------------+
|000000001052|    3|2019|    4|  8|        1293|
|000000001252|    2|2019|    4|  8|       12777|
|000000001501|    2|2019|    4|  8|      111384|
|000000001502|    5|2019|    4|  8|        4890|
|000000001831|    2|2019|    4|  8|        3955|
|000000020019|    4|2019|    4|  8|         470|
|000000020033|    2|2019|    4|  8|        7773|
|000000020062|    6|2019|    4|  8|        1640|
|000000020074|    2|2019|    4|  8|       30962|
|000000020086|    4|2019|    4|  8|         137|
|000000020087|    6|2019|    4|  8|        2131|
|000000020153|    5|2019|    4|  8|         149|
|000000020523|    6|2019|    4|  8|         918|
|000000200720|    2|2019|    4|  8|       13735|
|000000020073|    3|2019|    4|  8|         977|
|000000200171|    4|2019|    4|  8|         297|
|000000020254|    6|2019|    4|  8|         120|
|000000001051|    3|

+-----+-----------------+
|class|sum(VehicleCount)|
+-----+-----------------+
|    7|            38268|
|    3|           582241|
|    0|              359|
|    5|            86879|
|    6|           135452|
|    1|            26907|
|    4|            52577|
|    2|          5908182|
+-----+-----------------+

root
 |-- class: string (nullable = true)
 |-- sum(VehicleCount): long (nullable = true)

+------------+-----+----+-----+---+------------+
|       cosit|class|year|month|day|VehicleCount|
+------------+-----+----+-----+---+------------+
|000000001112|    3|2019|    4| 14|         958|
|000000001161|    2|2019|    4| 14|        2288|
|000000001401|    6|2019|    4| 14|         402|
|000000001592|    3|2019|    4| 14|         147|
|000000020013|    4|2019|    4| 14|         432|
|000000020114|    6|2019|    4| 14|         170|
|000000020526|    2|2019|    4| 14|       10418|
|000000020621|    2|2019|    4| 14|        3464|
|000000200811|    3|2019|    4| 14|        3157|
|00000020